In [17]:
exec(open("base_creation.py").read())

In [128]:
import sqlite3
import csv

# connexion à la base de données
connexion = sqlite3.connect('db_test.db')

# ouvrir le fichier csv en lecture
testcsv = open('test.csv', 'r', newline='')

> csv.reader renvoie une itération. La sortie n'est pas une liste ou un autre type pratiquement exploitable.

## Récupération des login et mdp à partir de la sortie de csv.reader dans un tableau.

Première méthode en parcourant la sortie globale (les rangées du fichiers .csv sont comptées comme des sous-éléments de la sortie).

In [127]:
tableau_login_mdp = []
rangee_tableau = 0
for rangee in list(csv.reader(testcsv)):
    tableau_login_mdp.append([rangee[0], rangee[1]]) # ajout de login et mdp dans une nouvelle rangée
    rangee_tableau += 1
tableau_login_mdp

[['Lui', 'mdp_test'],
 ['Elle', 'test_mdp'],
 ['Arche', 'noe'],
 ['Titanic', 'triste'],
 ['Cyrus', 'le Grand'],
 ['Xerxès', 'le roi des rois']]

Deuxième méthode, sans itérer sur les éléments-mêmes de la sortie.

In [129]:
tableau_login_mdp = []
tableauCSV = list(csv.reader(testcsv)) # la sortie de csv.reader ne peut pas être appelée plusieurs fois, alors on la stocke dans une variable.
for rangee in range(len(tableauCSV)-1):
    tableau_login_mdp.append([tableauCSV[rangee][0], tableauCSV[rangee][1]])
tableau_login_mdp

[['Lui', 'mdp_test'],
 ['Elle', 'test_mdp'],
 ['Arche', 'noe'],
 ['Titanic', 'triste'],
 ['Cyrus', 'le Grand']]

### Insertion des login et mdp dans la base de données

Avec INSERT.

In [46]:
for rangee in tableau_login_mdp:
    connexion.execute("INSERT INTO test (login, mdp) VALUES (?, ?);", (rangee[0], rangee[1]))
connexion.execute("SELECT * FROM test").fetchall()

[('Lui', 'mdp_test', 'Il aime les choux.'),
 ('Elle', 'test_mdp', "Elle n'aime pas les choux."),
 ('Cyrus', 'à remplacer', None),
 ('Xerxès', 'à remplacer', None),
 ('Lui', 'mdp_test', None),
 ('Elle', 'test_mdp', None),
 ('Arche', 'noe', None),
 ('Titanic', 'triste', None),
 ('Cyrus', 'le Grand', None),
 ('Xerxès', 'le roi des rois', None)]

### mise à jour des login et mdp dans la base de données

Avec UPDATE.

In [84]:
for loginSQL in connexion.execute("SELECT login, mdp FROM test").fetchall():
    for loginCSV in tableau_login_mdp:
        # login est un tuple, contenant seulement le login, au format ('login', ).
        if loginSQL[0] == loginCSV[0] and loginSQL[1] != loginCSV[1]: # Si le mdp SQL est différent du mdp CSV
            connexion.execute("UPDATE test SET mdp = ? WHERE mdp = ?", (loginCSV[1], loginSQL[1])) # Remplacer le mdp SQL par le mdp CSV

In [85]:
connexion.execute("SELECT login, mdp FROM test").fetchall()

[('Lui', 'mdp_test'),
 ('Elle', 'test_mdp'),
 ('Cyrus', 'le Grand'),
 ('Xerxès', 'le Grand')]

In [86]:
connexion.commit()
connexion.close()